In [1]:
import os
import re
import pandas as pd

RAW_PATH = "../data/books_with_genres.csv"
CLEAN_PATH = "../data/clean_books.csv"

In [2]:
df = pd.read_csv(RAW_PATH)
print(df.shape)
df.head()

(11127, 13)


,Book Id,Title,Author,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,genres
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Adve..."


In [3]:
def normalize_whitespace(text: str) -> str:
    if not isinstance(text, str):
        return ""
    return re.sub(r"\s+", " ", text).strip()


def clean_title(title: str) -> str:
    return normalize_whitespace(title)


def clean_author(author: str) -> str:
    return normalize_whitespace(author)


def clean_genres(genres: str) -> str:
    """
    Raw genres look like:
    'Fantasy;Young Adult;Fiction;Fantasy,Magic;Children...'
    We will:
      - split on ';'
      - lowercase
      - strip spaces
      - deduplicate while preserving order
    """
    if not isinstance(genres, str) or not genres.strip():
        return ""

    parts = [g.strip().lower() for g in genres.split(";") if g.strip()]
    seen = set()
    unique = []
    for g in parts:
        if g not in seen:
            seen.add(g)
            unique.append(g)
    return ", ".join(unique)  


In [4]:
df["Title"] = df["Title"].astype(str).apply(clean_title)
df["Author"] = df["Author"].astype(str).apply(clean_author)
df["genres_clean"] = df["genres"].apply(clean_genres)

df["average_rating"] = pd.to_numeric(df["average_rating"], errors="coerce")
df["num_pages"] = pd.to_numeric(df["num_pages"], errors="coerce")
df["ratings_count"] = pd.to_numeric(df["ratings_count"], errors="coerce")
df["text_reviews_count"] = pd.to_numeric(df["text_reviews_count"], errors="coerce")


In [5]:
def extract_year(date_str: str):
    """
    publication_date looks like '9/1/2004'
    We'll try to pull out the year.
    """
    if not isinstance(date_str, str):
        return None
    m = re.search(r"(\d{4})", date_str)
    if m:
        return int(m.group(1))
    return None

df["year"] = df["publication_date"].apply(extract_year)


In [6]:
before = len(df)

mask = df["Title"].notna() & df["Author"].notna()
mask &= df["Title"].str.len() > 0
mask &= df["Author"].str.len() > 0

mask &= df["average_rating"].notna()
mask &= df["genres_clean"].str.len() > 0

df = df[mask].copy()

after = len(df)
print(f"Filtered {before - after} rows, remaining {after}")


Filtered 97 rows, remaining 11030


In [7]:
before = len(df)

df = (
    df.sort_values(["average_rating", "ratings_count"], ascending=False)
      .drop_duplicates(subset=["Title", "Author"], keep="first")
)

after = len(df)
print(f"Deduplicated {before - after} rows, remaining {after}")


Deduplicated 320 rows, remaining 10710


In [8]:
def make_retrieval_text(row):
    parts = []

    title = row.get("Title")
    author = row.get("Author")
    genres = row.get("genres_clean")
    rating = row.get("average_rating")
    year = row.get("year")
    publisher = row.get("publisher")

    if pd.notna(title):
        parts.append(f"title: {title}.")
    if pd.notna(author):
        parts.append(f"author: {author}.")
    if isinstance(genres, str) and genres:
        parts.append(f"genres: {genres}.")
    if pd.notna(rating):
        parts.append(f"average rating: {rating}.")
    if pd.notna(year):
        parts.append(f"year: {int(year)}.")
    if pd.notna(publisher):
        parts.append(f"publisher: {publisher}.")

    return " ".join(parts).strip()


df["retrieval_text"] = df.apply(make_retrieval_text, axis=1)

df[["Title", "Author", "genres_clean", "average_rating", "year", "retrieval_text"]].head()


,Title,Author,genres_clean,average_rating,year,retrieval_text
4790,The Diamond Color Meditation: Color Pathway to...,John Diamond,"health, fitness",5.0,2006,title: The Diamond Color Meditation: Color Pat...
855,Literature Circle Guide: Bridge to Terabithia:...,Tara MacCarthy,education,5.0,2002,title: Literature Circle Guide: Bridge to Tera...
1243,Middlesex Borough (Images of America: New Jersey),Middlesex Borough Heritage Committee,history,5.0,2003,title: Middlesex Borough (Images of America: N...
4126,Zone of the Enders: The 2nd Runner Official St...,Tim Bogenn,games,5.0,2003,title: Zone of the Enders: The 2nd Runner Offi...
5649,Winchester Shotguns,Dennis Adler/R.L. Wilson,"antiques, collectibles",5.0,2008,title: Winchester Shotguns. author: Dennis Adl...


In [9]:
os.makedirs(os.path.dirname(CLEAN_PATH), exist_ok=True)

df.to_csv(CLEAN_PATH, index=False)
print("Saved cleaned dataset to:", CLEAN_PATH)


Saved cleaned dataset to: ../data/clean_books.csv


In [10]:
df = pd.read_csv("../data/clean_books.csv")
df.head()


,Book Id,Title,Author,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,genres,genres_clean,year,retrieval_text
0,17224,The Diamond Color Meditation: Color Pathway to...,John Diamond,5.0,1890995525,9781890995522,eng,74,5,3,2/1/2006,Square One Publishers,Health;Fitness,"health, fitness",2006,title: The Diamond Color Meditation: Color Pat...
1,2843,Literature Circle Guide: Bridge to Terabithia:...,Tara MacCarthy,5.0,0439271711,9780439271714,eng,32,4,1,1/1/2002,Teaching Resources,Education,education,2002,title: Literature Circle Guide: Bridge to Tera...
2,4287,Middlesex Borough (Images of America: New Jersey),Middlesex Borough Heritage Committee,5.0,0738511676,9780738511672,eng,128,2,0,3/17/2003,Arcadia Publishing,History,history,2003,title: Middlesex Borough (Images of America: N...
3,14741,Zone of the Enders: The 2nd Runner Official St...,Tim Bogenn,5.0,0744002354,9780744002355,eng,128,2,0,3/6/2003,BradyGames,Games,games,2003,title: Zone of the Enders: The 2nd Runner Offi...
4,20954,Winchester Shotguns,Dennis Adler/R.L. Wilson,5.0,0785821082,9780785821083,eng,372,2,0,5/15/2008,Chartwell Books,Antiques;Collectibles,"antiques, collectibles",2008,title: Winchester Shotguns. author: Dennis Adl...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10710 entries, 0 to 10709
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             10710 non-null  int64  
 1   Title               10710 non-null  object 
 2   Author              10710 non-null  object 
 3   average_rating      10710 non-null  float64
 4   isbn                10710 non-null  object 
 5   isbn13              10710 non-null  int64  
 6   language_code       10710 non-null  object 
 7   num_pages           10710 non-null  int64  
 8   ratings_count       10710 non-null  int64  
 9   text_reviews_count  10710 non-null  int64  
 10  publication_date    10710 non-null  object 
 11  publisher           10710 non-null  object 
 12  genres              10710 non-null  object 
 13  genres_clean        10710 non-null  object 
 14  year                10710 non-null  int64  
 15  retrieval_text      10710 non-null  object 
dtypes: f

In [12]:
len(df)

10710

In [13]:
df[["Title", "Author"]].duplicated().sum()

np.int64(0)

In [14]:
df["genres_clean"].sample(10)

1986     thriller, adventure, fiction, action, mystery,...
6908     classics, fiction, short stories, literature,1...
10193    fiction, childrens, fantasy, childrens,picture...
3010     classics, mystery, fiction, mystery,crime, mys...
4441     mystery, fiction, classics, mystery,crime, thr...
10706                                            education
5239     sequential art,comics, sequential art,graphic ...
3250     fiction, historical,historical fiction, myster...
7869     womens fiction,chick lit, fiction, romance, co...
7615     history, nonfiction, cultural,ireland, europea...
Name: genres_clean, dtype: object

In [15]:
df["retrieval_text"].sample(3).tolist()

['title: Aeschylus 2: The Persians/Seven Against Thebes/The Suppliants/Prometheus Bound. author: Aeschylus/David R. Slavitt/Smith Palmer Bovie. genres: classics, plays, drama, fiction, fantasy,mythology, poetry, literature, cultural,greece, plays,theatre, literature,ancient. average rating: 4.1. year: 1998. publisher: University of Pennsylvania Press.',
 'title: Twister. author: Michael Crichton/Anne-Marie Martin. genres: fiction, thriller, science fiction, adventure, media tie in, culture,film, adult fiction, romance, environment,nature, thriller,mystery thriller. average rating: 3.63. year: 1999. publisher: Ballantine Books.',
 'title: Wicked: The Grimmerie. author: David Cote/Stephen Schwartz/Joan Marcus/Winnie Holzman. genres: fantasy, fiction, plays, plays,theatre, music,musicals, paranormal,witches, music, drama, reference, science fiction fantasy. average rating: 4.18. year: 2005. publisher: Hachette Books.']

In [16]:
df["retrieval_text"].str.len().describe()

count    10710.000000
mean       280.501027
std         57.238993
min        110.000000
25%        252.000000
50%        284.000000
75%        313.000000
max       1011.000000
Name: retrieval_text, dtype: float64

In [17]:
df = pd.read_csv("../data/clean_books.csv")

if "genres" in df.columns:
    df = df.drop(columns=["genres"])

df = df.rename(columns={"genres_clean": "genres"})

df.to_csv("../data/clean_books.csv", index=False)

df.head()

,Book Id,Title,Author,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,genres,year,retrieval_text
0,17224,The Diamond Color Meditation: Color Pathway to...,John Diamond,5.0,1890995525,9781890995522,eng,74,5,3,2/1/2006,Square One Publishers,"health, fitness",2006,title: The Diamond Color Meditation: Color Pat...
1,2843,Literature Circle Guide: Bridge to Terabithia:...,Tara MacCarthy,5.0,0439271711,9780439271714,eng,32,4,1,1/1/2002,Teaching Resources,education,2002,title: Literature Circle Guide: Bridge to Tera...
2,4287,Middlesex Borough (Images of America: New Jersey),Middlesex Borough Heritage Committee,5.0,0738511676,9780738511672,eng,128,2,0,3/17/2003,Arcadia Publishing,history,2003,title: Middlesex Borough (Images of America: N...
3,14741,Zone of the Enders: The 2nd Runner Official St...,Tim Bogenn,5.0,0744002354,9780744002355,eng,128,2,0,3/6/2003,BradyGames,games,2003,title: Zone of the Enders: The 2nd Runner Offi...
4,20954,Winchester Shotguns,Dennis Adler/R.L. Wilson,5.0,0785821082,9780785821083,eng,372,2,0,5/15/2008,Chartwell Books,"antiques, collectibles",2008,title: Winchester Shotguns. author: Dennis Adl...


In [18]:
os.makedirs(os.path.dirname(CLEAN_PATH), exist_ok=True)

df.to_csv(CLEAN_PATH, index=False)
print("Saved cleaned dataset to:", CLEAN_PATH)

Saved cleaned dataset to: ../data/clean_books.csv
